In [1]:
# data analysis
import numpy as np
import pandas as pd
import sys
import glob

# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
matplotlib.rcParams['figure.figsize'] = [8,6]

In [2]:
datadir = '/home/lukas/projects/antibiotics/experiments/200129_AllData/'


############################
### compute phi          ###
############################

mu_tau_prod  = 1.
mu_tau_count = 0
mu0_sum      = 0.
mu0_count    = 0

methods = ['std','binarized','nonbinarized']
experiments = ['andy','philip']
for e in experiments:
    #print('\n***** ' + e + ' ******')
    for method in methods:
        mu_tau_prod  = 1.
        mu_tau_count = 0
        mu0_sum      = 0.
        mu0_count    = 0

        for fn in glob.glob(datadir + e + '/' + e + '_*/estimateMSP.'+method+'*'):
            a = np.genfromtxt(fn)
            if len(a) > 0:
                if ('WT' in fn and e == 'andy') or ('None' in fn and e == 'philip'):
                    mu0_sum   += np.sum(a[:,1])
                    mu0_count += len(a[:,1])

                mu_tau_prod   *= np.prod(np.sqrt(a[:,1]) / a[:,3])
                mu_tau_count  += len(a[:,1])

        mu0 = mu0_sum/mu0_count
        phi = np.power(mu_tau_prod,1./mu_tau_count)/np.sqrt(mu0)    

        #print('\nmethod = {}'.format(method))
        print('#{} mu0_{} = {:.6e}'.format(method,e,mu0))
        print('#{} phi_{} = {:.6e}'.format(method,e,phi))

#std mu0_andy = 5.250480e-02
#std phi_andy = 2.874982e+06
#binarized mu0_andy = 5.456231e-02
#binarized phi_andy = 2.762913e+06
#nonbinarized mu0_andy = 7.745263e-02
#nonbinarized phi_andy = 2.499300e+06
#std mu0_philip = 5.803148e-02
#std phi_philip = 1.294178e+06
#binarized mu0_philip = 5.661529e-02
#binarized phi_philip = 1.293733e+06
#nonbinarized mu0_philip = 8.491951e-02
#nonbinarized phi_philip = 8.298516e+05


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: genfromtxt: Empty input file: "/home/lukas/projects/antibiotics/experiments/200129_AllData/andy/andy_WT/estimateMSP.std.-IPTG"
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: genfromtxt: Empty input file: "/home/lukas/projects/antibiotics/experiments/200129_AllData/andy/andy_Triple/estimateMSP.std.-IPTG"
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: genfromtxt: Empty input file: "/home/lukas/projects/antibiotics/experiments/200129_AllData/andy/andy_WT/estimateMSP.binarized.-IPTG"
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: genfromtxt: Empty input file: "/home/lukas/projects/antibiotics/experiments/200129_AllData/andy/andy_Triple/estimateMSP.binarized.-IPTG"
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: genfromtxt: Empty input file: "/home/lukas/projects/antibiotics/experiments/200129_AllData/andy/andy_WT/estimate